First load everything necessary

In [3]:
import torch
import transformers
import automated_paper_analyser
import requests

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
torch.cuda.device_count()  # print 

1

In [4]:
! nvidia-smi

Mon Aug 19 21:43:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:15:00.0 Off |                  Off |
| 30%   40C    P8              24W / 300W |  25727MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
if torch.cuda.is_available():
    torch.set_default_device("cuda")
    print("CUDA")
else:
    torch.set_default_device("cpu")
    print("CPU")

CUDA


Define function to reconstruct the abstract from inverted index storage in OpenAlex

In [6]:
def reconstruct_text(inverted_index):
    word_index = [] 
    for k,v in inverted_index.items(): 
        for index in v: 
            word_index.append([k,index])
            
    word_index = sorted(word_index,key = lambda x : x[1])
    
    word_list = []
    for i in range(len(word_index)):
        word_list.append(word_index[i][0])
        
 
    separator = ' ' 
    reconstructed_text = separator.join(word_list) 

    return reconstructed_text

Define function to search OpenAlex

In [7]:
def search_openalex(search_phrase, result_count=10, min_year='2013'):
    base_url = "https://api.openalex.org/works"  # Replace with the actual API endpoint
    
    # Create filters
    filters = [
        "has_abstract:true",
        "has_fulltext:true",
        f"from_publication_date:{min_year}-01-01"
    ]
    
    # Construct the query parameters
    params = {
    "search": search_phrase,
    "filter": str.join(",", filters),  # Only return works with abstracts
    "per_page": result_count,  # Limit the search
    }
    
    r = requests.get(base_url, params=params)
    res_json = r.json()
    
    abstract_list = []
    for i in range(len(res_json["results"])):
        abstract_list.append(reconstruct_text(res_json["results"][i]['abstract_inverted_index']))
        
    return res_json["results"], abstract_list

Do a test search and investigate results

In [8]:
search_phrase = 'depression randomized control trial'
number_of_abstracts = 1

In [9]:
res_, abstract = search_openalex(search_phrase, number_of_abstracts)

In [10]:
res_[0]['title']

'A Randomized Controlled Trial of the Tumor Necrosis Factor Antagonist Infliximab for Treatment-Resistant Depression'

In [11]:
abstract[0]

'<h3>Context</h3>Increased concentrations of inflammatory biomarkers predict antidepressant nonresponse, and inflammatory cytokines can sabotage and circumvent the mechanisms of action of conventional antidepressants.<h3>Objectives</h3>To determine whether inhibition of the inflammatory cytokine tumor necrosis factor (TNF) reduces depressive symptoms in patients with treatment-resistant depression and whether an increase in baseline plasma inflammatory biomarkers, including high-sensitivity C-reactive protein (hs-CRP), TNF, and its soluble receptors, predicts treatment response.<h3>Design</h3>Double-blind, placebo-controlled, randomized clinical trial.<h3>Setting</h3>Outpatient infusion center at Emory University in Atlanta, Georgia.<h3>Participants</h3>A total of 60 medically stable outpatients with major depression who were either on a consistent antidepressant regimen (n = 37) or medication-free (n = 23) for 4 weeks or more and who were moderately resistant to treatment as determine

__Run various paper analyzer setups__

Version 1: Giving the system message once

In [12]:
automated_paper_analyser.analyse_initial_sys_message("meta-llama/Meta-Llama-3-8B", search_phrase, number_of_abstracts=1)

AttributeError: module 'transformers' has no attribute 'AutoModelForCausalLM'

Version 2: Giving the system message again for each paper

In [12]:
automated_paper_analyser.analyse_repeated_sys_message("meta-llama/Meta-Llama-3-8B", search_phrase, number_of_abstracts=1)

RuntimeError: Failed to import transformers.models.llama.modeling_llama because of the following error (look up to see its traceback):
/data/JH/miniconda3/envs/llms/lib/python3.11/site-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c104impl3cow11cow_deleterEPv

Version 3: Give instructions stepwise. First, ask for a summary (not shown) and then for this summary to be itemized.

In [13]:
automated_paper_analyser.analyse_stepwise("meta-llama/Meta-Llama-3-8B", search_phrase, number_of_abstracts=1)

RuntimeError: Failed to import transformers.models.llama.modeling_llama because of the following error (look up to see its traceback):
/data/JH/miniconda3/envs/llms/lib/python3.11/site-packages/flash_attn_2_cuda.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c104impl3cow11cow_deleterEPv